# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import tensorflow as tf
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf =tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

#tf.set_random_seed(seed)
tf.compat.v1.set_random_seed(seed)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

K.set_session(sess)
#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Activation, Dropout, Layer
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import optimizers,initializers,constraints,regularizers
from keras import backend as K
from keras.callbacks import LambdaCallback,ModelCheckpoint
from keras.utils import plot_model

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

import h5py
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import time
import pandas as pd
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"../Defined")
import Functions as F

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# 2. Loading data

In [2]:
train_data_frame=np.array(pd.read_csv('../Dataset/isolet1+2+3+4.data',header=None))
test_data_frame=np.array(pd.read_csv('../Dataset/isolet5.data',header=None))

train_data_arr=(train_data_frame[:,0:617]).copy()
train_label_arr=((train_data_frame[:,617]).copy()-1)
test_data_arr=(test_data_frame[:,0:617]).copy()
test_label_arr=((test_data_frame[:,617]).copy()-1)

data_arr=MinMaxScaler(feature_range=(0,1)).fit_transform(np.r_[train_data_arr,test_data_arr])

label_arr_onehot=np.r_[train_label_arr,test_label_arr]#to_categorical(train_label_arr)

In [3]:
key_feture_number=50

# 3. Calculation

In [4]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe

#--------------------------------------------------------------------------------------------------------------------------------       
def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR
 
#--------------------------------------------------------------------------------------------------------------------------------       
def cal(p_data_arr,\
        p_label_arr_onehot,\
        p_key_feture_number,\
        p_epochs_number,\
        p_batch_size_value,\
        p_is_use_bias,\
        p_seed):
    
    C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(p_data_arr,p_label_arr_onehot,test_size=0.2,random_state=p_seed)
    x_train,x_validate,y_train_onehot,y_validate_onehot= train_test_split(C_train_x,C_train_y,test_size=0.1,random_state=p_seed)
    x_test=C_test_x
    y_test_onehot=C_test_y

    os.environ['PYTHONHASHSEED'] = str(p_seed)
    np.random.seed(p_seed)
    rn.seed(p_seed)
    tf.compat.v1.set_random_seed(p_seed)

    #--------------------------------------------------------------------------------------------------------------------------------
    class Feature_Select_Layer(Layer):
    
        def __init__(self, output_dim, l1_lambda, **kwargs):
            super(Feature_Select_Layer, self).__init__(**kwargs)
            self.output_dim = output_dim
            self.l1_lambda=l1_lambda

        def build(self, input_shape):
            self.kernel = self.add_weight(name='kernel',\
                                          shape=(input_shape[1],),\
                                          initializer=initializers.RandomUniform(minval=0.999999, maxval=0.9999999, seed=p_seed),\
                                          trainable=True,\
                                          regularizer=regularizers.l1(self.l1_lambda),\
                                          constraint=constraints.NonNeg())
            
            super(Feature_Select_Layer, self).build(input_shape)
    
        def call(self, x, selection=False,k=p_key_feture_number):
            kernel=self.kernel        
            if selection:
                kernel_=K.transpose(kernel)
                kth_largest = tf.math.top_k(kernel_, k=k)[0][-1]
                kernel = tf.where(condition=K.less(kernel,kth_largest),x=K.zeros_like(kernel),y=kernel)        
            return K.dot(x, tf.linalg.tensor_diag(kernel))

        def compute_output_shape(self, input_shape):
            return (input_shape[0], self.output_dim)
    
    #--------------------------------------------------------------------------------------------------------------------------------
    def Fractal_Autoencoder(p_data_feature,\
                            p_feture_number,\
                            p_encoding_dim,\
                            p_learning_rate,\
                            p_l1_lambda,\
                            p_loss_weight_1,\
                            p_loss_weight_2,\
                            p_seed):
    
        input_img = Input(shape=(p_data_feature,), name='autoencoder_input')

        feature_selection = Feature_Select_Layer(output_dim=p_data_feature,\
                                                 l1_lambda=p_l1_lambda,\
                                                 input_shape=(p_data_feature,),\
                                                 name='feature_selection')

        feature_selection_score=feature_selection(input_img)
        feature_selection_choose=feature_selection(input_img,selection=True,k=p_feture_number)

        encoded = Dense(p_encoding_dim,\
                        activation='linear',\
                        kernel_initializer=initializers.glorot_uniform(p_seed),\
                        use_bias=p_is_use_bias,\
                        name='autoencoder_hidden_layer')
    
        encoded_score=encoded(feature_selection_score)
        encoded_choose=encoded(feature_selection_choose)
    
        bottleneck_score=encoded_score
        bottleneck_choose=encoded_choose
    
        decoded = Dense(p_data_feature,\
                        activation='linear',\
                        kernel_initializer=initializers.glorot_uniform(p_seed),\
                        use_bias=p_is_use_bias,\
                        name='autoencoder_output')
    
        decoded_score =decoded(bottleneck_score)
        decoded_choose =decoded(bottleneck_choose)

        latent_encoder_score = Model(input_img, bottleneck_score)
        latent_encoder_choose = Model(input_img, bottleneck_choose)
        feature_selection_output=Model(input_img,feature_selection_choose)
        autoencoder = Model(input_img, [decoded_score,decoded_choose])
    
        autoencoder.compile(loss=['mean_squared_error','mean_squared_error'],\
                            loss_weights=[p_loss_weight_1, p_loss_weight_2],\
                            optimizer=optimizers.Adam(lr=p_learning_rate))
    
        #print('Autoencoder Structure-------------------------------------')
        #autoencoder.summary()
        return autoencoder,feature_selection_output,latent_encoder_score,latent_encoder_choose
    
    t_start = time.time()
    #--------------------------------------------------------------------------------------------------------------------------------
    F_AE,\
    feature_selection_output,\
    latent_encoder_score_F_AE,\
    latent_encoder_choose_F_AE=Fractal_Autoencoder(p_data_feature=x_train.shape[1],\
                                                   p_feture_number=p_key_feture_number,\
                                                   p_encoding_dim=p_key_feture_number,\
                                                   p_learning_rate= 1E-3,\
                                                   p_l1_lambda=0.1,\
                                                   p_loss_weight_1=1,\
                                                   p_loss_weight_2=2,\
                                                   p_seed=p_seed)
    
    #--------------------------------------------------------------------------------------------------------------------------------
    F_AE_history = F_AE.fit(x_train, [x_train,x_train],\
                            epochs=p_epochs_number,\
                            batch_size=p_batch_size_value,\
                            shuffle=True,\
                            verbose=0,\
                            validation_data=(x_validate, [x_validate,x_validate]))
    
    t_used=time.time() - t_start
    
    write_to_csv(np.array([t_used]),"./log/FAE_time_bias_"+str(p_is_use_bias)+".csv")
    #--------------------------------------------------------------------------------------------------------------------------------
    p_data=F_AE.predict(x_test)
    numbers=x_test.shape[0]*x_test.shape[1]
    
    print("Completed on "+str(p_seed)+"!")
    print("MSE for one-to-one map layer",np.sum(np.power(np.array(p_data)[0]-x_test,2))/numbers)
    print("MSE for feature selection layer",np.sum(np.power(np.array(p_data)[1]-x_test,2))/numbers)
    
    FS_layer_output=feature_selection_output.predict(x_test)
    key_features=F.top_k_keepWeights_1(F_AE.get_layer(index=1).get_weights()[0],key_feture_number)

    #--------------------------------------------------------------------------------------------------------------------------------
    # Identify features successfully
    if np.sum(F_AE.get_layer(index=1).get_weights()[0]>0)>0:
        # Classification on original features
        train_feature=C_train_x
        train_label=C_train_y
        test_feature=C_test_x
        test_label=C_test_y
        orig_train_acc,orig_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
        # Classification on selected features
        selected_position_list=np.where(key_features>0)[0]
    
        train_feature_=np.multiply(C_train_x, key_features)
        train_feature=F.compress_zero_withkeystructure(train_feature_,selected_position_list)
        train_label=C_train_y

        test_feature_=np.multiply(C_test_x, key_features)
        test_feature=F.compress_zero_withkeystructure(test_feature_,selected_position_list)
        test_label=C_test_y
        selec_train_acc,selec_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
        # Linear reconstruction
        train_feature_=np.multiply(C_train_x, key_features)
        C_train_selected_x=F.compress_zero_withkeystructure(train_feature_,selected_position_list)

        test_feature_=np.multiply(C_test_x, key_features)
        C_test_selected_x=F.compress_zero_withkeystructure(test_feature_,selected_position_list)

        train_feature_tuple=(C_train_selected_x,C_train_x)
        test_feature_tuple=(C_test_selected_x,C_test_x)

        reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
    
        print("Classification on original data",orig_test_acc)
        print("Classification on selected features",selec_test_acc)
        print("Linear reconstruction loss",reconstruction_loss)
        print("-----------------------------------------------------------------------------")
        print("\n\n")
    
    else:
        orig_train_acc=-1
        orig_test_acc=-1
        selec_train_acc=-1
        selec_test_acc=-1
        reconstruction_loss=-1
        
    results=np.array([orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss])
    write_to_csv(results.reshape(1,len(results)),"./log/FAE_results_bias_"+str(p_is_use_bias)+".csv")
    
    return orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss

In [5]:
epochs_number=1000
batch_size_value=128
p_is_use_bias=False

p_data_arr=data_arr
p_label_arr_onehot=label_arr_onehot
p_key_feture_number=key_feture_number
p_epochs_number=epochs_number
p_batch_size_value=batch_size_value

In [ ]:
for p_seed in np.arange(0,50):
    orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss=cal(p_data_arr,\
                                                                                        p_label_arr_onehot,\
                                                                                        p_key_feture_number,\
                                                                                        p_epochs_number,\
                                                                                        p_batch_size_value,\
                                                                                        p_is_use_bias,\
                                                                                        p_seed)